In [1]:
from pyspark.sql import SparkSession
spark = SparkSession\
  .builder\
  .appName("SDG_Chapter15")\
  .getOrCreate()
sc = spark.sparkContext

In [3]:
df1 = spark.range(2, 1000000, 2)
df2 = spark.range(2, 1000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")
step4.explain()

== Physical Plan ==
*(4) HashAggregate(keys=[], functions=[sum(id#14L)])
+- Exchange SinglePartition
   +- *(3) HashAggregate(keys=[], functions=[partial_sum(id#14L)])
      +- *(3) Project [id#14L]
         +- *(3) BroadcastHashJoin [id#14L], [id#10L], Inner, BuildRight
            :- *(3) Project [(id#8L * 5) AS id#14L]
            :  +- Exchange RoundRobinPartitioning(5)
            :     +- *(1) Range (2, 1000000, step=2, splits=2)
            +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]))
               +- Exchange RoundRobinPartitioning(6)
                  +- *(2) Range (2, 1000000, step=4, splits=2)
